# Diffusion Models - Intuition and simple denoising autoencoder demo

This notebook explains the diffusion idea and provides a tiny denoising autoencoder demo (educational, not production).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib inline

# The demo will use MNIST if available; if not, create tiny synthetic images
try:
    (x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
    x = x_train.astype('float32') / 255.0
    x = np.expand_dims(x[:2000], -1)
except Exception as e:
    print('MNIST not available, creating synthetic data', e)
    x = np.random.rand(2000, 28, 28, 1)

# Add noise and train a small autoencoder to remove it (toy example)
noise_factor = 0.5
x_noisy = x + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x.shape)
x_noisy = np.clip(x_noisy, 0., 1.)

inp = layers.Input(shape=(28,28,1))
encoded = layers.Conv2D(32, 3, activation='relu', padding='same')(inp)
encoded = layers.MaxPooling2D(2, padding='same')(encoded)
encoded = layers.Conv2D(32, 3, activation='relu', padding='same')(encoded)
encoded = layers.MaxPooling2D(2, padding='same')(encoded)

decoded = layers.Conv2D(32, 3, activation='relu', padding='same')(encoded)
decoded = layers.UpSampling2D(2)(decoded)
decoded = layers.Conv2D(32, 3, activation='relu', padding='same')(decoded)
decoded = layers.UpSampling2D(2)(decoded)
decoded = layers.Conv2D(1, 3, activation='sigmoid', padding='same')(decoded)

auto = keras.Model(inp, decoded)
auto.compile(optimizer='adam', loss='binary_crossentropy')

auto.fit(x_noisy, x, epochs=5, batch_size=64, validation_split=0.1)

# Show examples
recon = auto.predict(x_noisy[:10])
plt.figure(figsize=(8,4))
for i in range(5):
    plt.subplot(2,5,i+1)
    plt.imshow(x_noisy[i].squeeze(), cmap='gray')
    plt.axis('off')
    plt.subplot(2,5,5+i+1)
    plt.imshow(recon[i].squeeze(), cmap='gray')
    plt.axis('off')
plt.suptitle('Noisy (top) vs Reconstructed (bottom)')
plt.show()

**Note:** This is an educational denoising example to illustrate the idea behind diffusion (iteratively removing noise). Full diffusion models are more complex and require special libraries.